In [141]:
%matplotlib inline

import os 
import librosa
import importlib
import matplotlib
import numpy as np
import string as strp
import pr_util as util
import matplotlib.pyplot as plt
import generate_global_features_2 as ggf

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn import svm, neighbors
from sklearn.naive_bayes import GaussianNB

importlib.reload(util)
importlib.reload(ggf)

<module 'generate_global_features_2' from '/var/tmp/ff/tcc/code/generate_global_features_2.py'>

In [142]:
num_species      = 3
n_min_per_specie = 20
n_max_per_specie = 250
cv = 10
n_global = 4 # number of global functions
scoring  = 'f1_weighted'
song_or_call = 'song'

#data_dirs = util.choose_species(num_species, n_min_per_specie, n_max_per_specie, song_or_call)
#data_dirs = util.return_n_most_frequent_species(num_species, song_or_call)
data_dirs = ['/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-2/Gnorimopsar chopi/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Sittasomus griseicapillus/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-2/Lathrotriccus euleri/']
print(data_dirs)
print(len(data_dirs))

#data_dirs = ['/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Synallaxis spixi/', 
#             '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Trogon surrucura/', 
#             '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Vanellus chilensis/']

#data_dirs = ['/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-1/Basileuterus leucoblepharus/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Theristicus caudatus/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-2/Gnorimopsar chopi/']
#data_dirs_pulse = util.dirs_to_pulse_dirs(data_dirs)


#print(data_dirs_pulse)
#data_dirs_pulse = ['/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Pulsos_Parte-4/Synallaxis spixi/', 
#                 '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Pulsos_Parte-4/Trogon surrucura/', 
#                 '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Pulsos_Parte-4/Vanellus chilensis/']



#data_dirs = ['/Users/felipefelix/USP/tcc/dataset/pr_article/syllables/experimentos_100/Synallaxis spixi/', 
#             '/Users/felipefelix/USP/tcc/dataset/pr_article/syllables/experimentos_100/Trogon surrucura/', 
#             '/Users/felipefelix/USP/tcc/dataset/pr_article/syllables/experimentos_100/Vanellus chilensis/']

['/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-2/Gnorimopsar chopi/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-4/Sittasomus griseicapillus/', '/var/tmp/ff/tcc/dataset/pr_article/S_A_C_Base_Parte-2/Lathrotriccus euleri/']
3


In [143]:
print(util.FEATURES)
feat = util.FEATURES[0]
feat = 'syllable_dur_list'
#labels_dict_p, labels_p, data_p = ggf.generate_global_features(n_global, feat, data_dirs_pulse, song_or_call, util.GLOBAL_FUNCTIONS, version = None)

labels_dict_o, labels_o, data_o = ggf.generate_global_features(n_global, feat, data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = 'filtered3')
#labels_dict_f1, labels_f1, data_f1 = ggf.generate_global_features(n_global, feat, data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = 'filtered1')
#labels_dict_f2, labels_f2, data_f2 = ggf.generate_global_features(n_global, feat, data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = 'filtered2')
#labels_dict_f3, labels_f3, data_f3 = ggf.generate_global_features(n_global, feat, data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = 'filtered3')
#labels_dict_f4, labels_f4, data_f4 = ggf.generate_global_features(n_global, feat, data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = 'filtered4')


# MULTI-FEATURES

#labels_dict_o, labels_o, data_o = ggf.generate_global_multi_features(n_global, ['spec_band', 'spec_cent'], data_dirs, song_or_call, util.GLOBAL_FUNCTIONS, version = None)


['syllable_dur']
number of files loaded: 169


In [144]:

print(labels_dict_o)
#print(labels_dict_p)
# print(labels_dict_f1)
# print(labels_dict_f2)
# print(labels_dict_f3)
#print(labels_dict_f4)



print(labels_o)
#print(labels_p)


# print(labels_f1)
# print(labels_f2)
# print(labels_f3)
#print(labels_f4)

print(len(labels_o))
#print(len(labels_p))

# print(len(labels_f1))
# print(len(labels_f2))
# print(len(labels_f3))
#print(len(labels_f4))

print(data_o.shape)
print(data_o[:10])
#print(data_p[:10])
#print(data_f4[:10])

# print(data_f1[:10])
# print(data_f2[:10])
# print(data_f3[:10])

print(len(data_o))
#print(len(data_p))
#print(len(data_f4))
# print(len(data_f1))
# print(len(data_f2))
# print(len(data_f3))


{'Lathrotriccus Euleri': 2, 'Gnorimopsar Chopi': 0, 'Sittasomus Griseicapillus': 1}
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
169
(169, 4)
[[ 0.09755102  0.03012353  0.16630385  0.0723356 ]
 [ 0.04552154  0.0033993   0.04825397  0.0399093 ]
 [ 0.05238095  0.          0.05238095  0.05238095]
 [ 0.08269841  0.01636838  0.11034014  0.06780045]
 [ 0.09397833  0.02200073  0.13124717  0.07179138]
 [ 0.0821542   0.00605442  0.08820862  0.07609977]
 [ 0.05166667  0.01445049  0.07297052  0.03587302]
 [ 0.05018141  0.0253288   0.0755102   0.02485261]
 [ 0.04439909  0.00680272  0.05120181  0.03759637]
 [ 0.05843537  0.00269841  0.06113379  0.05573696]]
169


In [ ]:
util.plot_scatter(data_o[:,0], data_o[:,1], labels_o, feat + ' mean', feat + ' std')

In [ ]:
util.plot_scatter(data_f4[:,0], data_f4[:,1], labels_f4, feat + ' mean', feat + ' std')

In [ ]:
util.plot_scatter(data_f1[:,2], data_f1[:,3], labels_f1, feat + ' mean', feat + ' std')

In [ ]:
util.plot_scatter(data_f2[:,2], data_f2[:,3], labels_f2, feat + ' mean', feat + ' std')

In [137]:
def generate_scores(clf, clf_name, data, labels, scoring, cv = cv):
    scores  = cross_val_score(clf, data, labels, n_jobs = -1, cv = cv, scoring = scoring, verbose = 3)
    print(scores)
    print('{0} - {1}: {2:.2f} (+/- {3:.2f})'.format(clf_name, scoring, scores.mean(), scores.std() * 2))


In [138]:
clf     = neighbors.KNeighborsClassifier(3, weights = 'uniform')

generate_scores(clf, 'kNN', data_o, labels_o, scoring)

#generate_scores(clf, 'kNN', data_p, labels_p, scoring)
#generate_scores(clf, 'kNN', data_f1, labels_f1, scoring)
#generate_scores(clf, 'kNN', data_f2, labels_f2, scoring)
#generate_scores(clf, 'kNN', data_f3, labels_f3, scoring)
#generate_scores(clf, 'kNN', data_f4, labels_f4, scoring)


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=0.2948717948717949, total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] ...................... , score=0.25925925925925924, total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] ...................... , score=0.40774410774410774, total=   0.0s
[CV] ....................... , score=0.3218954248366013, total=   0.0s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


[ 0.25925926  0.29487179  0.40774411  0.32189542  0.22491349  0.35773601
  0.37431319  0.31993007  0.45148601  0.37878788]
kNN - f1_weighted: 0.34 (+/- 0.13)


In [139]:
clf     = GaussianNB()

generate_scores(clf, 'gNB', data_o, labels_o, scoring)
#generate_scores(clf, 'gNB', data_p, labels_p, scoring)
#generate_scores(clf, 'gNB', data_f1, labels_f1, scoring)
#generate_scores(clf, 'gNB', data_f2, labels_f2, scoring)
#generate_scores(clf, 'gNB', data_f3, labels_f3, scoring)
#generate_scores(clf, 'gNB', data_f4, labels_f4, scoring)


[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ...................... , score=0.33660130718954245, total=   0.0s
[CV] ....................... , score=0.4122899159663866, total=   0.0s


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ...................... , score=0.12626262626262624, total=   0.0s


...

/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ...................... , score=0.07894736842105264, total=   0.0s
[CV] ....................... , score=0.2048611111111111, total=   0.0s
[CV] .................................... , score=0.275, total=   0.0s


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ....................... , score=0.2986111111111111, total=   0.0s
[ 0.12626263  0.1010101   0.22150072  0.22150072  0.33660131  0.41228992
  0.275       0.20486111  0.07894737  0.29861111]
gNB - f1_weighted: 0.23 (+/- 0.20)


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [140]:
clf = svm.SVC(kernel = 'linear', max_iter=-1, C = 1, decision_function_shape='ovr')

generate_scores(clf, 'SVM', data_o, labels_o, scoring)
#generate_scores(clf, 'SVM', data_p, labels_p, scoring)
#generate_scores(clf, 'SVM', data_f1, labels_f1, scoring)
#generate_scores(clf, 'SVM', data_f2, labels_f2, scoring)
#generate_scores(clf, 'SVM', data_f3, labels_f3, scoring)
#generate_scores(clf, 'SVM', data_f4, labels_f4, scoring)



[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV]  ................................................................
[CV]  ................................................................


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ....................... , score=0.2177777777777778, total=   0.0s
[CV]  ................................................................
[CV] ....................... , score=0.2177777777777778, total=   0.0s


...

/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ...................... , score=0.20454545454545453, total=   0.0s
[CV] ...................... , score=0.20454545454545453, total=   0.0s
[CV] ...................... , score=0.20454545454545453, total=   0.0s


/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/gradmac/fsfelix/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[CV] ...................... , score=0.20454545454545453, total=   0.0s
[CV] ...................... , score=0.20454545454545453, total=   0.0s
[ 0.21777778  0.21777778  0.32208995  0.32208995  0.24019608  0.20454545
  0.20454545  0.20454545  0.20454545  0.20454545]
SVM - f1_weighted: 0.23 (+/- 0.09)


[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
